In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

# Reference existing Delta table by path
delta_path = "dbfs:/Volumes/workspace/ecommerce/ecommerce_data/events"
deltaTable = DeltaTable.forPath(spark, delta_path)

In [0]:
# Create incremental data
updates = (
    spark.read.format("delta").load(delta_path)
    .sample(0.02, seed=42)          # small batch
    .withColumn("price", F.col("price") + 5)  # simulate change
)

     

In [0]:

# Merge
deltaTable.alias("t").merge(
    updates_clean.alias("s"),
    """
    t.user_id = s.user_id
    AND t.event_time = s.event_time
    AND t.product_id = s.product_id
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

print("Incremental MERGE completed")

Incremental MERGE completed


In [0]:
# Optimize
spark.sql("OPTIMIZE events_table ZORDER BY (event_type, user_id)")
spark.sql("VACUUM events_table RETAIN 168 HOURS")

DataFrame[path: string]